In [10]:
import pyomo.environ as pyo
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash import Dash, dcc, html
import webbrowser
from threading import Timer
import dash_bootstrap_components as dbc
import plotly.io as pio
from bs4 import BeautifulSoup
from dash.dependencies import Input, Output
import json

# Read price data from Excel
price_data = pd.read_csv('price_data.csv')
price = price_data['Price'].values  # Assuming the second column is named 'Price'


In [2]:
# Constants
hours = len(price)  # Should be 8760 for a full year
max_charge_rate = 1  # kW
max_discharge_rate = 1  # kW
battery_capacity = 4  # kWh
initial_soc = 2  # kWh
final_soc = 2  # kWh
charge_efficiency = 0.9
discharge_efficiency = 0.9

# Create the model
model = pyo.ConcreteModel()

# Time index
model.T = pyo.RangeSet(0, hours - 1)

# Variables
model.charge = pyo.Var(model.T, within=pyo.NonNegativeReals, bounds=(0, max_charge_rate))
model.discharge = pyo.Var(model.T, within=pyo.NonNegativeReals, bounds=(0, max_discharge_rate))
model.soc = pyo.Var(model.T, within=pyo.NonNegativeReals, bounds=(0, battery_capacity))

# Objective function: Maximize revenue
def revenue_rule(model):
    return sum(price[t] * (model.discharge[t] * discharge_efficiency - model.charge[t] / charge_efficiency) for t in model.T)
model.revenue = pyo.Objective(rule=revenue_rule, sense=pyo.maximize)

# Constraints
def soc_rule(model, t):
    if t == 0:
        return model.soc[t] == initial_soc + model.charge[t] * charge_efficiency - model.discharge[t] / discharge_efficiency
    else:
        return model.soc[t] == model.soc[t-1] + model.charge[t] * charge_efficiency - model.discharge[t] / discharge_efficiency
model.soc_constraint = pyo.Constraint(model.T, rule=soc_rule)

# Final state of charge constraint
model.final_soc_constraint = pyo.Constraint(expr=model.soc[hours-1] == final_soc)

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model, tee=True)

# Extract results
charge_schedule = [pyo.value(model.charge[t]) for t in model.T]
discharge_schedule = [pyo.value(model.discharge[t]) for t in model.T]
soc_schedule = [pyo.value(model.soc[t]) for t in model.T]

# Convert results to DataFrame
results_df = pd.DataFrame({
    'Timestamp': price_data['Timestamp'],
    'Price': price,
    'Charge': charge_schedule,
    'Discharge': discharge_schedule,
    'SOC': soc_schedule
})


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmp_th95nvv.glpk.raw
 --wglp /var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmp40caz6re.glpk.glp
 --cpxlp /var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmpoluq6ud4.pyomo.lp
Reading problem data from '/var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmpoluq6ud4.pyomo.lp'...
8761 rows, 26280 columns, 35040 non-zeros
105128 lines were read
Writing problem data to '/var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmp40caz6re.glpk.glp'...
113882 lines were written
GLPK Simplex Optimizer 5.0
8761 rows, 26280 columns, 35040 non-zeros
Preprocessing...
8760 rows, 17519 columns, 26278 non-zeros
Scaling...
 A: min|aij| =  9.000e-01  max|aij| =  1.000e+00  ratio =  1.111e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 8760
      0: obj =   1.730808000e+02 inf =   2.889e+00 (2)
      7: obj =   1.510616272e+02 inf 

In [3]:
model.revenue() * 20

772576.1123514402

In [4]:
695/772

0.9002590673575129

In [27]:
# Create interactive plot for charging and discharging
fig_charge_discharge = go.Figure()
fig_charge_discharge.add_trace(go.Scatter(x=results_df['Timestamp'], y=results_df['Charge'], mode='lines', name='Charge'))
fig_charge_discharge.add_trace(go.Scatter(x=results_df['Timestamp'], y=results_df['Discharge'], mode='lines', name='Discharge'))
fig_charge_discharge.update_layout(
    title='Charging and Discharging Schedule',
    xaxis_title='Time',
    yaxis_title='Power (kW)',
    xaxis=dict(
        tickmode='linear',
        dtick=24 * 30,  # 86400000.0 milliseconds in a day * 30 days
    ),
    legend=dict(x=1.05, y=1),  # Position the legend outside the plot
    autosize=True,
    margin=dict(l=40, r=40, t=40, b=40),
)

# Create interactive plot for state of charge
fig_soc = go.Figure()
fig_soc.add_trace(go.Scatter(x=results_df['Timestamp'], y=results_df['SOC'], mode='lines', name='State of Charge', line=dict(color='green')))
fig_soc.update_layout(
    title='State of Charge over Time',
    xaxis_title='Time',
    yaxis_title='State of Charge (kWh)',
    xaxis=dict(
        tickmode='linear',
        dtick=24.0 * 30,  # 86400000.0 milliseconds in a day * 30 days
    ),
    legend=dict(x=1.05, y=1),  # Position the legend outside the plot
    autosize=True,
    margin=dict(l=40, r=40, t=40, b=40),
)

fig_charge_discharge.show()
fig_soc.show()

In [31]:
# JavaScript for downloading CSV
csv_download_js = """
function download_csv(data, filename) {
    var csv = 'Timestamp,Price,Charge,Discharge,SOC\\n';
    data.forEach(function(row) {
        csv += row.join(',');
        csv += "\\n";
    });

    var hiddenElement = document.createElement('a');
    hiddenElement.href = 'data:text/csv;charset=utf-8,' + encodeURI(csv);
    hiddenElement.target = '_blank';
    hiddenElement.download = filename;
    hiddenElement.click();
}
"""

# JavaScript for linking plots
link_plots_js = """
function linkPlots(plot1, plot2) {
    plot1.on('plotly_relayout', function(eventdata) {
        Plotly.relayout(plot2, eventdata);
    });
    plot2.on('plotly_relayout', function(eventdata) {
        Plotly.relayout(plot1, eventdata);
    });
}

document.addEventListener('DOMContentLoaded', function() {
    var plot1 = document.getElementById('plot1');
    var plot2 = document.getElementById('plot2');
    linkPlots(plot1, plot2);
});
"""

# Create the combined layout
combined_html = f"""
<html>
    <head>
        <title>BESS Revenue Simulation Dashboard</title>
        <style>
            .responsive-plot {{
                width: 80%;
                max-width: 3600px;
                height: 80%;
                max-height: 600px;
                margin: auto;
            }}
        </style>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <script>{csv_download_js}</script>
        <script>{link_plots_js}</script>
    </head>
    <body>
        <h1>BESS Revenue Simulation Dashboard</h1>
        <div>
            <h3>Charging and Discharging Schedules</h3>
            <div id="plot1" class="responsive-plot">
                {pio.to_html(fig_charge_discharge, include_plotlyjs=False, full_html=False)}
            </div>
        </div>
        <div>
            <h3>State of Charge Over Time</h3>
            <div id="plot2" class="responsive-plot">
                {pio.to_html(fig_soc, include_plotlyjs=False, full_html=False)}
            </div>
        </div>
    </body>
</html>
"""

# Save the combined HTML to a file
with open("bess_dashboard.html", "w") as f:
    f.write(combined_html)